<a href="https://colab.research.google.com/github/janwaelty/Master-Thesis/blob/main/visual_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Networks and the Art Market: A Deep Learning Approach to Valuation**


In [4]:

import os
import shutil
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.stats import linregress
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torchvision import datasets, transforms
import shutil
import os
from google.colab import drive
import zipfile

# Define source and destination paths
source_path = "/content/drive/My Drive/Colab Notebooks/visual_model.ipynb"
destination_path = "/content/drive/My Drive/visual_model.ipynb"

  # Check if the source file exists before moving
if os.path.exists(source_path):
  # Move the file
  shutil.move(source_path, destination_path)
  print(f"Notebook moved to: {destination_path}")

!cp /content/drive/My\ Drive/data_preprocess.py /content/

from data_preprocess import (
    data_split as data_split,
    fit_visualization as fit_visualization,
    repeated_to_single_transactions as repeated_to_single_transactions,
    one_hot as one_hot,
    standardization as standardization,
    data_filter as data_filter,
    add_prev_avg_price as add_prev_avg_price,
    standardization as standardization
)





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from PIL import Image
import torch
from torchvision import transforms
import os



class MixedTransactionDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, transform=None):
        # Ensure numerical data is in float32 format
        self.X = X.reset_index(drop=True).values.astype('float32')  # Convert X to numpy array
        self.y = y.reset_index(drop=True).values.astype('float32')  # Convert y to numpy array
        self.image_paths = X['image_url'].values

        # Drop the 'image_url' column from X after extracting the paths
        self.X = X.drop(columns=['image_url']).reset_index(drop=True).values.astype('float32')

        self.transform = transform

    def __getitem__(self, index):
        # Get numerical features (X) and target values (y)
        x = torch.tensor(self.X[index], dtype=torch.float32)
        y = torch.tensor(self.y[index], dtype=torch.float32)

        # Get the image path (image_url) and open the image
        image_path = self.image_paths[index]


        # If it's a local path, open the image directly
        image = Image.open(image_path)

        # Apply any transformations if provided (resize, normalization, etc.)
        if self.transform:
            image = self.transform(image)

        return x, y, image

    def __len__(self):
        return len(self.X)





**Load zip file with images**

In [5]:

# Pfad zur ZIP-Datei auf Google Drive
zip_path = "/content/drive/My Drive/pic.zip"
extract_path = "/content/data"  # Zielordner für die entpackten Bilder

# Entpacken und die '_MACOSX' Ordner überspringen
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Liste der Dateien im ZIP-Archiv
    file_list = zip_ref.namelist()

    # Filtern der _MACOSX Ordner und unnötiger Dateien
    file_list = [f for f in file_list if not f.startswith('__MACOSX')]

    # Entpacken der relevanten Dateien
    zip_ref.extractall(extract_path, members=file_list)

# Überprüfen, ob das Entpacken erfolgreich war
print(f"Dateien entpackt in: {extract_path}")

Dateien entpackt in: /content/data


In [ ]:
"""
2import os
import shutil
from google.colab import drive

# Google Drive mounten
drive.mount('/content/drive')

# Pfad zur ZIP-Datei auf Google Drive
zip_path = "/content/drive/My Drive/pic.zip"
extract_path = "/content/data/pic"  # Zielordner für die entpackten Bilder
output_path = "/content/data/all_images"  # Zielordner für alle Bilder

# Entpacken der ZIP-Datei (wie vorher)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    file_list = [f for f in file_list if not f.startswith('__MACOSX')]  # Überspringen von _MACOSX
    zip_ref.extractall(extract_path, members=file_list)

# Ordner erstellen, in dem alle Bilder zusammenkommen
os.makedirs(output_path, exist_ok=True)

# Alle Bilder aus den Unterordnern zusammenführen
for subfolder in os.listdir(extract_path):
    subfolder_path = os.path.join(extract_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Alle Dateien im Unterordner auflisten
        for filename in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, filename)
            if os.path.isfile(file_path):
                # Datei in den gemeinsamen Ordner verschieben
                shutil.move(file_path, os.path.join(output_path, filename))

# Überprüfen, ob alles erfolgreich verschoben wurde
print(f"Alle Bilder befinden sich jetzt im Ordner: {output_path}")
print(f"Verzeichnisinhalt: {os.listdir(output_path)}")


In [ ]:
if __name__ == 'main':
  # Mount Google Drive
  drive.mount('/content/drive')